<a href="https://colab.research.google.com/github/daemonX10/Data-Science/blob/main/Vgg_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle datasets download -d nunenuh/pytorch-challange-flower-dataset

Dataset URL: https://www.kaggle.com/datasets/nunenuh/pytorch-challange-flower-dataset
License(s): CC0-1.0
 99% 328M/330M [00:18<00:00, 25.0MB/s]
100% 330M/330M [00:18<00:00, 18.5MB/s]


In [17]:
import zipfile
zip_ref = zipfile.ZipFile('/content/pytorch-challange-flower-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [10]:
import tensorflow as tf
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [36]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/dataset/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

Found 6552 files belonging to 102 classes.


In [37]:
validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/dataset/valid',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

Found 818 files belonging to 102 classes.


In [19]:
validation_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
# # Assuming 'process' function is defined elsewhere and responsible for
# # resizing images, ensure it resizes images to (224, 224)

# def process(image, label):
#   # ... other preprocessing steps
#   image = tf.image.resize(image, (224, 224))  # Resize image to match model input
#   return image, label

# train_ds = train_ds.map(process)
# validation_ds = validation_ds.map(process)

# # ... rest of your model definition

In [22]:
import tensorflow

In [38]:
def process(image,label):
   image = tensorflow.cast(image/224.,tensorflow.float32)
   return image,label

In [39]:
train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [49]:
# Vgg with some modification

In [57]:
from warnings import filters
from keras.regularizers import l2
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),activation='relu',kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))


model.add(Conv2D(filters=128,kernel_size=(3,3),kernel_regularizer=l2(0.01),padding='same',activation='relu'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=256,kernel_size=(3,3),kernel_regularizer=l2(0.01),padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=512,kernel_size=(3,3),kernel_regularizer=l2(0.01),padding='same',activation='relu'))
model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2))


model.add(Flatten())
model.add(Dense(units=4096,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=4096,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=102,activation='softmax'))

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[86528,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2] name: 

In [52]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 222, 222, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_10 (Conv2D)          (None, 222, 222, 64)      36928     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 111, 111, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 111, 111, 128)     73856     
                                                                 
 conv2d_12 (Conv2D)          (None, 111, 111, 128)    

In [53]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [55]:
history = model.fit(train_ds,epochs=5,validation_data=validation_ds)

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node sequential_4/conv2d_10/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib

In [45]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [46]:
## Vgg Pretrained
import gdown
url = "https://drive.google.com/file/d/12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP/view?usp=sharing"
file_id = url.split("/")[-2]
print(file_id)
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "catdog.zip")

12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP


Downloading...
From: https://drive.google.com/uc?/export=download&id=12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP
To: /content/catdog.zip
100%|██████████| 9.09M/9.09M [00:00<00:00, 37.0MB/s]


'catdog.zip'

In [47]:
!unzip catdog.zip

Archive:  catdog.zip
   creating: train/
   creating: train/Cat/
  inflating: train/Cat/0.jpg         
  inflating: train/Cat/1.jpg         
  inflating: train/Cat/2.jpg         
  inflating: train/Cat/cat.2405.jpg  
  inflating: train/Cat/cat.2406.jpg  
  inflating: train/Cat/cat.2436.jpg  
  inflating: train/Cat/cat.2437.jpg  
  inflating: train/Cat/cat.2438.jpg  
  inflating: train/Cat/cat.2439.jpg  
  inflating: train/Cat/cat.2440.jpg  
  inflating: train/Cat/cat.2441.jpg  
  inflating: train/Cat/cat.2442.jpg  
  inflating: train/Cat/cat.2443.jpg  
  inflating: train/Cat/cat.2444.jpg  
  inflating: train/Cat/cat.2445.jpg  
  inflating: train/Cat/cat.2446.jpg  
  inflating: train/Cat/cat.2447.jpg  
  inflating: train/Cat/cat.2448.jpg  
  inflating: train/Cat/cat.2449.jpg  
  inflating: train/Cat/cat.2450.jpg  
  inflating: train/Cat/cat.2451.jpg  
  inflating: train/Cat/cat.2452.jpg  
  inflating: train/Cat/cat.2453.jpg  
  inflating: train/Cat/cat.2454.jpg  
  inflating: train/Cat/

In [48]:
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Flatten

In [ ]:
epoch = 25
base_model =  VGG16(input_shape=(224,224,3),include_top=False,weights='imagent')
base_model.trainable = False

model = keras.Sequential()
model.add(base_model)
